In [1]:
import requests
import json
import os
import glob
import fnmatch
import pandas as pd
import markdown
from html import unescape
from bs4 import BeautifulSoup
from haystack import Document
from haystack.nodes import PreProcessor

/scratch/users/anup/miniconda3/envs/finetune-gllm/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-03-22 13:36:44,893] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [9]:
directory_path = "../../../gtn-data/"

def read_md_file_1(path):
    with open(path) as f:
        content = f.read()
        return content

def read_md_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        md_content = file.read()
    return extract_plain_text_from_md(md_content)

def extract_plain_text_from_md(md_content):
    html_content = markdown.markdown(md_content)
    plain_text = ''.join(BeautifulSoup(html_content, "html.parser").findAll(text=True))
    return plain_text.strip()

md_files = []
md_files_data = []
gtn_docs = []

#included_content = "/topics/statistics/tutorials/intro_deep_learning/"
included_content = "/topics/"

for root, dirs, files in os.walk(directory_path):
    for filename in files:
        if fnmatch.fnmatch(filename, '*.md'):
            path = os.path.join(root, filename)
            if included_content in path:
                s_path = path.split("/")[-3:]
                tutorial_name = "_".join(s_path)
                md_plain_text = read_md_file(path)
                md_files.append(tutorial_name)
                md_files_data.append(md_plain_text)
                pr_dict = {"content": md_plain_text, "meta": {"name": tutorial_name}}
                doc = Document.from_json(json.dumps(pr_dict))
                gtn_docs.append(doc)

df_gtn_md_content = pd.DataFrame(zip(md_files, md_files_data), columns=["path", "content"])
df_gtn_md_content

/tmp/ipykernel_107869/1729941110.py:15: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  plain_text = ''.join(BeautifulSoup(html_content, "html.parser").findAll(text=True))


,path,content
0,topics_admin_README.md,Admin\nPlease refer to the CONTRIBUTING.md bef...
1,topics_admin_index.md,layout: topic\ntopic_name: admin
2,admin_faqs_README.md,"FAQs\nFormerly known as snippets, FAQs provide..."
3,admin_faqs_admin-testing.md,title: Operating system compatibility\narea: a...
4,admin_faqs_ansible-connection.md,title: Variable connection\narea: ansible\nbox...
...,...,...
1359,tutorials_circos_tutorial.md,"layout: tutorial_hands_on\ntitle: ""Visualisati..."
1360,circos_faqs_index.md,layout: faq-page
1361,circos_workflows_index.md,layout: workflow-list
1362,tutorials_jbrowse_tutorial.md,layout: tutorial_hands_on\ntitle: Genomic Data...


In [10]:
df_gtn_md_content.to_csv("../out/df_gtn.tsv", sep="\t", index=None)

In [4]:
'''for i in range(n_pages):
    i += offset
    page_path = "https://api.github.com/repos/galaxyproject/galaxy/pulls?page={}&state=all&per_page=100".format(i)
    r = requests.get(page_path, allow_redirects=True, headers=headers)
    loaded_string = json.loads(r.content.decode("utf-8"))


print("Writing PRs to file...")
open(path + "github_pr_page_{}-{}.json".format(offset, offset+n_pages), 'w').write(json.dumps(prs))'''

'for i in range(n_pages):\n    i += offset\n    page_path = "https://api.github.com/repos/galaxyproject/galaxy/pulls?page={}&state=all&per_page=100".format(i)\n    r = requests.get(page_path, allow_redirects=True, headers=headers)\n    loaded_string = json.loads(r.content.decode("utf-8"))\n\n\nprint("Writing PRs to file...")\nopen(path + "github_pr_page_{}-{}.json".format(offset, offset+n_pages), \'w\').write(json.dumps(prs))'